In [66]:
import sys
import os
# Thêm thư mục gốc của project vào sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [67]:
from utils.api_client import RAGAPIClient
# Initialize API client
api_client = RAGAPIClient(base_url="http://localhost:8000")

# Check if API is available
if not api_client.check_health():
    print("⚠️ RAG API is not available at http://localhost:8000")
    print("Please start the RAG service first!")
else:
    print("✅ RAG API is available")

2025-11-14 15:04:16.955 | INFO     | utils.api_client:__init__:28 - Initialized RAG API Client: http://localhost:8000/api/v1


✅ RAG API is available


In [68]:
import json
file_path = "../data/questions.json"

# Đọc file JSON
with open(file_path, "r", encoding="utf-8") as f:
    questions = json.load(f)

# In ra dữ liệu đã load
print(questions)

[{'question_id': 'Q001', 'question': 'Ngành Kỹ thuật Máy tính có thể làm những công việc gì sau khi tốt nghiệp?'}, {'question_id': 'Q003', 'question': 'Sinh viên mới vào học nên theo thứ tự học những môn nền tảng nào trong hai năm đầu để có nền tảng vững về lập trình, cấu trúc dữ liệu, hệ điều hành và kiến trúc máy tính?'}, {'question_id': 'Q004', 'question': 'Nếu em muốn theo hướng Hệ thống nhúng/IoT thì yêu cầu học những môn gì và kế hoạch ra sao?'}, {'question_id': 'Q005', 'question': 'Ngành Kỹ thuật Máy tính có những định hướng chuyên ngành tự chọn nào?'}, {'question_id': 'Q006', 'question': ' Trong Ngành Công Nghệ Thông Tin, môn Thực tập doanh nghiệp có bắt buộc không? Có thể đổi sang hình thức khác không? '}, {'question_id': 'Q007', 'question': 'Chương trình Trí Tuệ Nhân Tạo có những phương thức tốt nghiệp nào? Thời lượng/ tín chỉ và điều kiện tham gia mỗi phương thức ra sao?'}]


In [70]:
rag_responses = []
# Generate answers for first N test cases
for i, tc in enumerate(questions):
    question_id = tc['question_id']
    question = tc['question']
    print(f"[{question_id}] Generating answer for: {question[:50]}...")
    
    try:
        # Call RAG API
        response = api_client.ask_question(question)
        formatted = api_client.format_response(response)
        
        # Store result
        rag_responses.append({
            'question_id':question_id,
            'question': question,
            'answer': formatted['answer'],
            'contexts': formatted['sources'],
            'processing_time': formatted['processing_time']
        })
        
        print(f"  ✅ Got answer ({len(formatted['answer'])} chars)")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        rag_responses.append({
            'question_id': tc['id'],
            'question': question,
            'answer': '',
            'contexts': [],
            'error': str(e)
        })

print(f"\n✅ Generated {len([r for r in rag_responses if r.get('answer')])} answers")

2025-11-14 15:04:49.250 | INFO     | utils.api_client:ask_question:50 - Asking question: Ngành Kỹ thuật Máy tính có thể làm những công việc...


[Q001] Generating answer for: Ngành Kỹ thuật Máy tính có thể làm những công việc...


2025-11-14 15:05:05.925 | INFO     | utils.api_client:ask_question:59 - Got response in 16.65s
2025-11-14 15:05:05.932 | INFO     | utils.api_client:ask_question:50 - Asking question: Sinh viên mới vào học nên theo thứ tự học những mô...


  ✅ Got answer (382 chars)
[Q003] Generating answer for: Sinh viên mới vào học nên theo thứ tự học những mô...


2025-11-14 15:05:24.907 | INFO     | utils.api_client:ask_question:59 - Got response in 18.94s
2025-11-14 15:05:24.924 | INFO     | utils.api_client:ask_question:50 - Asking question: Nếu em muốn theo hướng Hệ thống nhúng/IoT thì yêu ...


...

  ✅ Got answer (298 chars)
[Q006] Generating answer for:  Trong Ngành Công Nghệ Thông Tin, môn Thực tập doa...


2025-11-14 15:06:10.647 | INFO     | utils.api_client:ask_question:59 - Got response in 12.18s
2025-11-14 15:06:10.647 | INFO     | utils.api_client:ask_question:50 - Asking question: Chương trình Trí Tuệ Nhân Tạo có những phương thức...


  ✅ Got answer (52 chars)
[Q007] Generating answer for: Chương trình Trí Tuệ Nhân Tạo có những phương thức...


2025-11-14 15:06:22.200 | INFO     | utils.api_client:ask_question:59 - Got response in 11.53s


  ✅ Got answer (52 chars)

✅ Generated 6 answers


In [71]:
output_path = "../data/rag_results.json"

# Ghi ra JSON với định dạng đẹp
with open(output_path, "w", encoding="utf-8-sig") as f:
    json.dump(rag_responses, f, ensure_ascii=False, indent=2)

print(f"✅ Saved {len(rag_responses)} responses to {output_path}")

✅ Saved 6 responses to ../data/rag_results.json


In [72]:
import utils.convert_results_to_csv as convert_utils

In [73]:
convert_utils.convert_json_to_csv(
    json_file="../data/rag_results.json",
    csv_file="../data/open_rag_eval_results/rag_results.csv"
)

✅ Converted 6 queries to ../data/open_rag_eval_results/rag_results.csv
   Total rows: 30


In [64]:
import json
import csv
file_path = "../data/questions.json"

# Đọc file JSON
with open(file_path, "r", encoding="utf-8") as f:
    questions = json.load(f)

with open("questions.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)  # Bao quanh tất cả giá trị bằng ""
    writer.writerow(["query"])

    for row in questions:
        # Lấy nội dung câu hỏi và loại bỏ newline hoặc dấu đặc biệt gây lỗi
        question = row.get("question", "").replace("\n", " ").replace("\r", " ").strip()
        writer.writerow([question])

## RAPTOR

In [57]:
from utils.api_client_raptor_and_colbertv2 import RAGAPIClient
# Initialize API client
api_client = RAGAPIClient(base_url="http://localhost:8000")

# Check if API is available
if not api_client.check_health():
    print("⚠️ RAG API is not available at http://localhost:8000")
    print("Please start the RAG service first!")
else:
    print("✅ RAG API is available")

2025-11-14 14:30:42.527 | INFO     | utils.api_client_raptor_and_colbertv2:__init__:28 - Initialized RAG API Client: http://localhost:8000/api/v1


✅ RAG API is available


In [56]:
import json
file_path = "../data/questions.json"

# Đọc file JSON
with open(file_path, "r", encoding="utf-8") as f:
    questions = json.load(f)

# In ra dữ liệu đã load
print(questions)

[{'question_id': 'Q001', 'question': 'Ngành Kỹ thuật Máy tính có thể làm những công việc gì sau khi tốt nghiệp?'}, {'question_id': 'Q002', 'question': 'Chương trình Kỹ thuật Máy tính có bao nhiêu tín chỉ tổng cộng, phân bổ giữa các khối kiến thức chính như thế nào?'}, {'question_id': 'Q003', 'question': 'Sinh viên mới vào học nên theo thứ tự học những môn nền tảng nào trong hai năm đầu để có nền tảng vững về lập trình, cấu trúc dữ liệu, hệ điều hành và kiến trúc máy tính?'}, {'question_id': 'Q004', 'question': 'Nếu em muốn theo hướng Hệ thống nhúng/IoT thì yêu cầu học những môn gì và kế hoạch ra sao?'}, {'question_id': 'Q005', 'question': 'Ngành Kỹ thuật Máy tính có những định hướng chuyên ngành tự chọn nào?'}, {'question_id': 'Q006', 'question': ' Trong Ngành Công Nghệ Thông Tin, môn Thực tập doanh nghiệp có bắt buộc không? Có thể đổi sang hình thức khác không? '}, {'question_id': 'Q007', 'question': 'Chương trình Trí Tuệ Nhân Tạo có những phương thức tốt nghiệp nào? Thời lượng/ tín c

In [58]:
rag_responses = []
# Generate answers for first N test cases
for i, tc in enumerate(questions):
    question_id = tc['question_id']
    question = tc['question']
    print(f"[{question_id}] Generating answer for: {question[:50]}...")
    
    try:
        # Call RAG API
        response = api_client.ask_question(question)
        print(response)
        
        # Store result
        rag_responses.append({
            'question_id':question_id,
            'question': question,
            'answer': response['answer'],
            'sources': response['sources'],
            'contexts': [context.replace("\n", " ") for context in response.get('contexts', [])],
        })
        
        print(f"  ✅ Got answer ({len(response['answer'])} chars)")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        rag_responses.append({
            'question_id': tc['id'],
            'question': question,
            'answer': '',
            'contexts': [],
            'sources': [],
            'error': str(e)
        })

print(f"\n✅ Generated {len([r for r in rag_responses if r.get('answer')])} answers")

2025-11-14 14:30:44.066 | INFO     | utils.api_client_raptor_and_colbertv2:ask_question:46 - Asking question: Ngành Kỹ thuật Máy tính có thể làm những công việc...


[Q001] Generating answer for: Ngành Kỹ thuật Máy tính có thể làm những công việc...


2025-11-14 14:30:56.097 | INFO     | utils.api_client_raptor_and_colbertv2:ask_question:55 - Got response in 0.00s
2025-11-14 14:30:56.097 | INFO     | utils.api_client_raptor_and_colbertv2:ask_question:46 - Asking question: Chương trình Kỹ thuật Máy tính có bao nhiêu tín ch...


{'query': 'Ngành Kỹ thuật Máy tính có thể làm những công việc gì sau khi tốt nghiệp?', 'answer': 'Sau khi tốt nghiệp, sinh viên ngành Kỹ thuật Máy tính có thể đảm nhận nhiều công việc khác nhau, bao gồm:\n\n*   **Phát triển giải pháp phần mềm máy tính.**\n*   **Thiết kế các hệ thống số, hệ thống nhúng, hệ thống IoT và ứng dụng trí tuệ nhân tạo.**\n*   **Nghiên cứu và ứng dụng Công nghệ thông tin** tại các viện, trung tâm nghiên cứu và các trường đại học, cao đẳng.\n*   **Giảng dạy** các môn liên quan đến kỹ thuật máy tính tại các trường đại học, cao đẳng, trung học chuyên nghiệp, dạy nghề và các trường phổ thông.\n*   **Nghiên cứu khoa học** trong các lĩnh vực như phần mềm hệ thống, thiết kế hệ thống số, điều khiển tự động và các hệ thống nhúng tại các viện nghiên cứu, các trung tâm và cơ quan nghiên cứu.\n*   **Làm việc** trong các bộ phận công nghệ thông tin của các đơn vị như hành chính sự nghiệp, ngân hàng, viễn thông, hàng không, xây dựng,…\n*   **Làm việc** trong các công ty sản 

2025-11-14 14:31:03.063 | INFO     | utils.api_client_raptor_and_colbertv2:ask_question:55 - Got response in 0.00s
2025-11-14 14:31:03.064 | INFO     | utils.api_client_raptor_and_colbertv2:ask_question:46 - Asking question: Sinh viên mới vào học nên theo thứ tự học những mô...


...

{'query': 'Ngành Kỹ thuật Máy tính có những định hướng chuyên ngành tự chọn nào?', 'answer': 'Dựa trên thông tin được cung cấp, ngành Kỹ thuật Máy tính có những định hướng chuyên ngành tự chọn sau:\n\n*   **Hệ thống nhúng và IoT** (CE232, CE439, CE339, CE340, CE342, CE348, CE437, CE438)\n*   **Robotics và Trí tuệ nhân tạo** (CE233, CE347, CE440, CE320, CE406, CE441, CE442)\n\nNgoài ra, sinh viên có thể tự chọn các môn học từ khối kiến thức chuyên ngành của các CTĐT ngành KTMT ban hành trước năm 2024 hoặc các môn học mới (được ban hành từ năm 2024 trở về sau) có nội dung và số tín chỉ phù hợp để bổ sung kiến thức và xét tương đương cho các môn học chuyên đề này.', 'sources': [{'text': 'Sinh viên ngành Kỹ thuật Máy tính được yêu cầu tích lũy tối thiểu 18 tín chỉ chuyên ngành, bao gồm các môn bắt buộc và tự chọn. Các môn bắt buộc bao gồm CE201 (Đồ án 1) và CE206 (Đồ án 2). Sinh viên c...', 'level': 1, 'score': 0.01639344262295082, 'section_title': 'N/A', 'heading_path': 'N/A'}, {'text': '

2025-11-14 14:32:08.328 | INFO     | utils.api_client_raptor_and_colbertv2:ask_question:55 - Got response in 0.00s
2025-11-14 14:32:08.330 | INFO     | utils.api_client_raptor_and_colbertv2:ask_question:46 - Asking question: Chương trình Trí Tuệ Nhân Tạo có những phương thức...


{'query': ' Trong Ngành Công Nghệ Thông Tin, môn Thực tập doanh nghiệp có bắt buộc không? Có thể đổi sang hình thức khác không? ', 'answer': 'Dựa trên thông tin được cung cấp, chương trình đào tạo ngành Công nghệ Thông Tin **không có quy định cụ thể về môn Thực tập doanh nghiệp**. \n\nThông tin chỉ đề cập đến các hình thức tốt nghiệp như:\n\n*   Khóa luận tốt nghiệp (10 tín chỉ)\n*   Chuyên đề tốt nghiệp (4 tín chỉ) + Đồ án tốt nghiệp (6 tín chỉ)\n*   Đồ án tốt nghiệp tại doanh nghiệp (10 tín chỉ)\n\nDo đó, sinh viên có thể lựa chọn hình thức tốt nghiệp phù hợp với mình, và việc lựa chọn doanh nghiệp thực tập (nếu có) sẽ phụ thuộc vào hình thức tốt nghiệp được chọn.\n\n**Về việc đổi sang hình thức khác, thông tin không đề cập cụ thể.** Tuy nhiên, sinh viên có thể thảo luận với giảng viên hướng dẫn để điều chỉnh hình thức tốt nghiệp (nếu có thể) phù hợp với mục tiêu học tập và phát triển của bản thân.', 'sources': [{'text': 'Chương trình đào tạo ngành Công nghệ Thông Tin (hoặc Kỹ thuật 

2025-11-14 14:32:19.929 | INFO     | utils.api_client_raptor_and_colbertv2:ask_question:55 - Got response in 0.00s


{'query': 'Chương trình Trí Tuệ Nhân Tạo có những phương thức tốt nghiệp nào? Thời lượng/ tín chỉ và điều kiện tham gia mỗi phương thức ra sao?', 'answer': 'Chương trình Trí Tuệ Nhân Tạo có một phương thức tốt nghiệp chính, được xây dựng dựa trên học chế tín chỉ và đồ án tốt nghiệp tại doanh nghiệp.\n\n**Thời lượng/Tín chỉ:**\n\n*   Sinh viên cần tích lũy tối thiểu **128 tín chỉ** để được công nhận tốt nghiệp.\n*   Phân bổ tín chỉ theo các khối:\n    *   Giáo dục đại cương: 45 tín chỉ\n    *   Cơ sở ngành: 57 tín chỉ\n    *   Tự chọn ngành: 8 tín chỉ\n    *   Tự chọn liên ngành: 8 tín chỉ\n    *   Kiến thức tốt nghiệp: 10 tín chỉ\n\n**Điều kiện tham gia:**\n\n*   Hoàn thành ít nhất 128 tín chỉ, bao gồm tất cả các tín chỉ trong các khối đã nêu trên.\n*   Hoàn thành các môn học bắt buộc của chương trình.\n*   Thành công trong thực hiện đồ án tốt nghiệp tại doanh nghiệp (một yêu cầu quan trọng của chương trình).\n\nLưu ý rằng sinh viên cần đáp ứng các điều kiện chung của Trường Đại học Cô

In [59]:
output_path = "../data/rag_results_raptor.json"

# Ghi ra JSON với định dạng đẹp
with open(output_path, "w", encoding="utf-8-sig") as f:
    json.dump(rag_responses, f, ensure_ascii=False, indent=2)

print(f"✅ Saved {len(rag_responses)} responses to {output_path}")

✅ Saved 7 responses to ../data/rag_results_raptor.json


In [60]:
import utils.convert_results_to_csv as convert_utils

In [65]:
convert_utils.convert_json_to_csv_with_contexts(
    json_file="../data/rag_results_raptor.json",
    csv_file="../data/open_rag_eval_results_raptor/rag_results_raptor.csv"
)

✅ Converted 6 queries to ../data/open_rag_eval_results_raptor/rag_results_raptor.csv
   Total rows: 18


In [35]:
import pandas as pd

file_path = "../data/questions.csv"

# Đọc CSV
df = pd.read_csv(file_path)

# Hiển thị toàn bộ bảng
print(df)



                                               query
0  Ngành Kỹ thuật Máy tính có thể làm những công ...
1  Chương trình Kỹ thuật Máy tính có bao nhiêu tí...
2  Sinh viên mới vào học nên theo thứ tự học nhữn...
3  Nếu em muốn theo hướng Hệ thống nhúng/IoT thì ...
4  Ngành Kỹ thuật Máy tính có những định hướng ch...
5   Trong Ngành Công Nghệ Thông Tin, môn Thực tập...
6  Nếu em muốn theo hướng Hệ thống nhúng/IoT hoặc...
7  Chương trình Trí Tuệ Nhân Tạo có những phương ...
8  Ngành Kỹ Thuật Máy Tính học những môn toán, lý...
9  Ngành Kỹ thuật máy tính chương trình đại trà h...
